## Interactive plots in Jupyter notebooks with matplotlib and ipywidgets
## matplotlib and the inline backend

Author: DVC  
Date: 01/12/2020   


### Using the inline backend with a GUI ?

The matplotlib inline backend is described as a **non-GUI** backend.  So, why do we insist on using it with a (simple) GUI ?    
- it is the most commonly used backend 
- it is probably the only backend that behaves consistently across platforms  
- what works with the inline backend is almost guaranteed to work with any other backend
- we often need something more fancy than just typing in cells as provided natively in our Jupyter notebooks

What are the opportunities and limitations with the inline backend ?  
- the basic (ipy)widgets work fine in the inline backend, creating some possibilities
- figures are rendered as png's in the output cell and its components are not accessible after being rendered
- interactivity can only be simulated by redrawing the full figure after each change
    + this is not an issue with slow updates of the controls
    + flickering and visual glitches from the clear-and-redraw cycle are inevitable
    + computational load quickly increases for complex applications
    + be aware that a figure is NOT closed after rendering

In [1]:
%matplotlib widget

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, interactive_output
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import IPython.display as ipd
import time

In [3]:
# any of these could be useful during debugging, but not needed if all is running fine
#
#%matplotlib --list
#ipd.set_matplotlib_close(close=True)
#plt.close('all')

In [4]:
plt.close('all')
def my_sine(x, w, amp, phi):
    return amp*np.sin(w * (x-phi))
#
def synth(sigtype='sin', freq=200.0, Fs=8000, Tmax=0.5):
    t = np.linspace(0.0, Tmax, int(Tmax*Fs), endpoint=False)
    if sigtype == 'sin':
        x = np.sin(2.0*np.pi*freq*t)
    elif sigtype == 'square':
        x = signal.square(2.0*np.pi*freq*t)
    elif sigtype == 'sawtooth':
        x = signal.sawtooth(2.0*np.pi*freq*t)
    else:
        print( 'signal: Unrecognized signal type')
    return x, t

### 1. Using interact() for the most simple interfaces

In [5]:
# Methodology:
# ============
#  1. Create the Figure
#  2. Use interact() for the interactive Loop:
#       (a) CLEAR AXIS (or figure)
#       (b) PLOT AGAIN with each interaction
#
# REMARK: 
# (1)  AVOID following loop construction:
#      (a) create a figure
#      (b) draw to the figure
#   At first sight it behaves the same, but a new figure is created each time
#   and after some time you will run out of available figures as they are not closed automatically
# (2) for the inline backend it doesn't seem to matter if you use plt.cla() or plt.clf()
#     the widget backend is more reliable with plt.cla(), plus there is no accumulation of figures

x = np.linspace(0, 2 * np.pi, 100)
fig = plt.figure(figsize=(3,3))

@interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi, 0.01))
def update(w = 1.0, amp=1, phi=0):
    # clear the figure and start over
    # plt.cla()
    plt.clf()
    print(plt.gcf().number)
    plt.plot(x, my_sine(x, w, amp, phi), color='C0')
    plt.ylim([-4, 4])
    plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='w', max=10), FloatSlider(value=1.0, description='amp', m…

In [6]:
# The non interactive baseline
x = np.linspace(0, 2 * np.pi, 100)
fig, (ax1,ax2) = plt.subplots(2,figsize=(5,3))
ax1.plot(x, my_sine(x, 1., 1., 0.), color='C0')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.close('all')
box_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px',
        width = '50%',
        height = '500px',
        flex_shrink = 1
     )
button_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px',
        width = '50%',
        height = '100%',
        flex_shrink =2
     )
x = np.linspace(0, 2 * np.pi, 100)
#out = widgets.Output(layout=box_layout)
#comments = widgets.Output(layout=box_layout)
#comments.layout.width = '200px'
fig, (ax1,ax2) = plt.subplots(2,figsize=(5,3))
ax1.plot(x, my_sine(x, 1., 1., 0.), color='C0')
# update()
#global fig,ax
#fig,ax = plt.subplots(2,1,constrained_layout=True, figsize=(4, 4))
    
@interact(w=(0, 10, 1))
def update(w = 1.0):
    #global fig,ax
    # clear the figure and start over
    #if ax is not None:
    #    fig,ax = plt.subplots(2,1,constrained_layout=True, figsize=(4, 4))
    # ax = fig.add_subplots(2,1)
    # clear_output()
    ax1.cla()
    ax1.plot(x, my_sine(x, w, 1., 0.), color='C0')
    ax1.set_ylim([-4, 4])
    ax1.grid(True)
    #with comments:
    #    print('Fig Number:',fig.number)
    plt.show()
#widgets.HBox([out,comments])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='w', max=10), Output()), _dom_classes=('widget-interact',…

In [29]:
wg_freq = widgets.FloatSlider(value=100.0,step=10.,min=100.0,max=600.,description='Frequency')
box_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
UI = widgets.VBox( [wg_freq] ,layout=box_layout)
out = widgets.Output()
first_time = True
global fig, ax1, ax2
fig, (ax1,ax2) = plt.subplots(2,1,constrained_layout=True, figsize=(8, 4))
y,x = synth(sigtype='sin',freq=440.,Tmax=0.25)
ax1.plot(x, y, color='C0')
    
def mySine(freq=440.):
    global first_time, ax1, ax2, fig
    plt.sca(ax1)
    plt.cla()
    y,x = synth(sigtype='sin',freq=freq,Tmax=0.25)
    #y=amp*y
    ax1.plot(x, y, color='C0')
    ax1.set_ylim([-1, 1])
    ax1.grid(True)
    ax1.set_title('Waveform')
    ax1.set_xlabel('Time(sec)')
    # ax1.set_xlim([0,Tmax])
    
screen = widgets.interactive_output(mySine, {'freq': wg_freq})
   
widgets.VBox([  UI, screen ])
#layout=box_layout) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2. Using interactive_output() for more control over the layout 

In [9]:
# create the sliders for interaction
signal_types = [ 'sin', 'square', 'sawtooth' ]
wg_sigtype = widgets.Dropdown(options=signal_types,value='sin',description="Signal")
wg_amp = widgets.FloatSlider(value=1.0,step=0.05,max=1.0,description='Amplitude',continous_update=False)
wg_freq = widgets.FloatSlider(value=100.0,step=10.,min=100.0,max=600.,description='Frequency')
wg_Tmax = widgets.FloatSlider(value=0.2,step=0.05,min=0.1,max=.4,description='Duration',
                             continuous_update=False)
box_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
UI1 = widgets.VBox( [wg_sigtype, wg_amp, wg_freq, wg_Tmax] ,layout=box_layout)
# create two dedicated output widgets, one for audio and one for plotting
out_audio = widgets.Output()
out_comments = widgets.Output()
UI2 = widgets.VBox( [out_audio,out_comments] ,layout=box_layout)

first_time = True
# fig,ax = plt.subplots(2,1,constrained_layout=True, figsize=(8, 4))
    
def sin_gui1(amp=1., sigtype='sin', freq=440.0, Fs=8000, Tmax=1.0):

    global first_time, ax, fig
    if first_time:
        fig,ax = plt.subplots(2,1,constrained_layout=True, figsize=(8, 4))
        first_time = False
    else:
        #with out_comments:
        # print("clearing axis")
        #plt.sca(ax[0])
        #plt.cla()
        plt.sca(ax[1])
        plt.cla()
        
    y,x = synth(sigtype=sigtype,freq=freq,Tmax=Tmax)
    y=amp*y
    ax[0].plot(x, y, color='C0')
    ax[0].set_ylim([-1, 1])
    ax[0].grid(True)
    ax[0].set_title('Waveform')
    ax[0].set_xlabel('Time(sec)')
    ax[0].set_xlim([0,Tmax])
    freq,spec = signal.periodogram(y,fs=8000,scaling='spectrum')
    # freq,spec = signal.periodogram(1000.*y/np.sqrt(len(y)),fs=8000,scaling='spectrum')
    #specdb = 10.*np.log10(freq+1.e-7)
    ax[1].plot(freq,np.sqrt(2*spec))
    ax[1].set_title('Spectrum')
    ax[1].set_xlabel('Frequency')
    ax[1].set_ylim([0, 1])
    ax[1].grid(True)
    ax[1].set_xlim([0,4000])
    # here come the things that go to dedicated output widgets
    with out_audio:
        ipd.clear_output()
        try:
            with out_comments:
                ipd.clear_output()
                print("Audio playing unnormalized")
            ipd.display(ipd.Audio(data=y,rate=8000,normalize=False, autoplay=True))
        except: 
            try:
                with out_comments:
                    ipd.clear_output()
                    print("Warning: playing normalized sound")
                ipd.display(ipd.Audio(data=y,rate=8000, autoplay=True))
            except: pass


screen = widgets.interactive_output(sin_gui1, {'sigtype':wg_sigtype,'amp': wg_amp, 'freq': wg_freq, 'Tmax': wg_Tmax})
screen.layout = box_layout
   
widgets.HBox([ screen, widgets.VBox([ UI1, UI2 ], layout=box_layout) ])

In [10]:
# create the sliders for interaction
plt.close('all')
box_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )




class Audio_Demo(widgets.HBox):

    def __init__(self):
        super().__init__()
        
        self.amp = 1.
        self.sigtype = 'sin'
        self.freq = 440.
        self.samplerate = 8000
        self.Tmax = .25
        self.first_time = True
        
        signal_types = [ 'sin', 'square', 'sawtooth' ]
        self.wg_sigtype = widgets.Dropdown(options=signal_types,value='sin',description="Signal")
        self.wg_amp = widgets.FloatSlider(value=1.0,step=0.05,max=1.0,description='Amplitude',continous_update=False)
        self.wg_freq = widgets.FloatSlider(value=100.0,step=10.,min=100.0,max=600.,description='Frequency')
        self.UI = widgets.VBox( [self.wg_sigtype, self.wg_amp, self.wg_freq] ,layout=box_layout)

        self.wg_freq.observe(self.freq_observe,'value')
        
#        self.screen = widgets.interactive_output(self.sin_gui1, {'sigtype':self.wg_sigtype,'amp': self.wg_amp, 'freq': self.wg_freq})
        self.screen = widgets.Output()
        self.screen.layout = box_layout
        
        self.audio = widgets.Output()
        self.out_comments = widgets.Output()  

        self.sin_gui1()
        self.children = [ self.screen, widgets.VBox([self.UI, self.audio, self.out_comments]) ] 

#    def sin_gui1(self,amp=1., sigtype='sin', freq=440.0):
    def sin_gui1(self):

        Fs = self.samplerate
        Tmax = self.Tmax
        amp = self.amp
        sigtype = self.sigtype
        freq = self.freq

        with self.screen:
            if self.first_time:
                self.fig,self.ax = plt.subplots(2,1,constrained_layout=True, figsize=(8, 4))
                self.first_time = False
            else:
                plt.sca(self.ax[0])
                plt.cla()
                plt.sca(self.ax[1])
                plt.cla()  

            ax = self.ax
            y,x = synth(sigtype=sigtype,freq=freq,Tmax=Tmax)
            y=amp*y
            ax[0].plot(x, y, color='C0')
            ax[0].set_ylim([-1, 1])
            ax[0].grid(True)
            ax[0].set_title('Waveform')
            ax[0].set_xlabel('Time(sec)')
            ax[0].set_xlim([0,Tmax])
            freq,spec = signal.periodogram(y,fs=8000,scaling='spectrum')
            ax[1].plot(freq,np.sqrt(2*spec))
            ax[1].set_title('Spectrum')
            ax[1].set_xlabel('Frequency')
            ax[1].set_ylim([0, 1])
            ax[1].grid(True)
            ax[1].set_xlim([0,4000])
    # here come the things that go to dedicated output widgets

        with self.audio:
            ipd.clear_output()
            try:
                with self.out_comments:
                    ipd.clear_output()
                    print("Audio playing unnormalized")
                ipd.display(ipd.Audio(data=y,rate=8000,normalize=False, autoplay=True))
            except: 
                try:
                    with self.out_comments:
                        ipd.clear_output()
                        print("Warning: playing normalized sound")
                    ipd.display(ipd.Audio(data=y,rate=8000, autoplay=True))
                except: pass
     
    def freq_observe(self,change):
        self.freq = change.new
        # with self.screen:
        self.sin_gui1()
    
    
Audio_Demo()    

Audio_Demo(children=(Output(layout=Layout(border='solid 1px black', margin='0px 10px 10px 0px', padding='5px 5…

In [11]:
plt.close('all')

In [12]:
fig.show()

In [13]:
# Method 2: CLEAR selected axes and redraw with each operation (use OO API)
# BACKENDS: NOT for INLINE !
def my_sine(x, w, amp, phi):
    return amp*np.sin(w * (x-phi))
#
x = np.linspace(0, 2 * np.pi, 100)
fig, ax = plt.subplots(figsize=(3, 3))

@interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi, 0.01))
def update(w = 1.0, amp=1, phi=0):
    plt.cla()
    ax.plot(x, my_sine(x, w, amp, phi), color='C0')
    ax.set_ylim([-4, 4])
    ax.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='w', max=10), FloatSlider(value=1.0, description='amp', m…

In [14]:
# Method 3: Only update line data inside existing axis
# BACKENDS: NOT for INLINE !
# 

def my_sine(x, w, amp, phi):
    return amp*np.sin(w * (x-phi))
#
x = np.linspace(0, 2 * np.pi, 100)
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_ylim([-4, 4])
ax.grid(True)

@interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi, 0.01))
def update(w = 1.0, amp=1, phi=0):
    """Remove old lines from plot and plot new one"""
    [l.remove() for l in ax.lines]
    ax.plot(x, my_sine(x, w, amp, phi), color='C0')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='w', max=10), FloatSlider(value=1.0, description='amp', m…

## Updating the plot line data

In [15]:
# Method 3b: maintain the figure and just refresh the changed data
#    1) create a figure  
#    2)  just update the y_data if you may assume that the x_data hasn't changed; 
#          between calls 
#
# BACKENDS: all, except 'inline'  
# 

def my_sine(x, w, amp, phi):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return amp*np.sin(w * (x-phi))
x = np.linspace(0, 2 * np.pi, 100)

#
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_ylim([-4, 4])
ax.grid(True)
line, = ax.plot(x, my_sine(x, 0,0,0), color='C0')

@interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi+0.01, 0.01))
def update(w = 1.0, amp=1, phi=0):
    """change the ydata values """
    line.set_ydata( my_sine(x, w, amp, phi) )
    fig.canvas.draw()
    # fig.canvas.flush_events()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='w', max=10), FloatSlider(value=1.0, description='amp', m…

In [16]:
# a simple audio signal generator
# with control for type amplitude, phase ... 
#
# the audio is immediately generated and rendered
#   continuous update are used for frequency and amplitude
#   continuous_update=False is used for the duration widget
# !! setting continuous_update is the only 'easy' control over frequency of update with interact()
# !! more complex asynchronous implementations are suggested on the ipywidgets doc pages
# !! under 'debouncing' and 'throttling' .. but this complicates the code significantly
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html?highlight=continuous_update#Continuous-updates
# !! these solutions are not applicable to interact(), but require .observe() instead, see later
#
# as the x-axis needs to be modified with the duration
#  it is more appropriate to do full redraws at every iteration
#
# BACKENDS: all
#
samplerate=8000
def generate(type='sin', freq=200.0, Fs=samplerate, Tmax=0.5):
    t = np.linspace(0.0, Tmax, int(Tmax*Fs), endpoint=False)
    if type == 'sin':
        x = np.sin(2.0*np.pi*freq*t)
    elif type == 'square':
        x = signal.square(2.0*np.pi*freq*t)
    elif type == 'sawtooth':
        x = signal.sawtooth(2.0*np.pi*freq*t)
    else:
        print( 'signal: Unrecognized signal type')
    return x, t

signal_types = [ 'sin', 'square', 'sawtooth' ]
# alternative to Dropdown is to use RadioButtons
wg_signal = widgets.Dropdown(options=signal_types,
                             value='sin',description="Signal")
wg_amp = widgets.FloatSlider(value=0.5,step=0.05,max=1.0,description='Amplitude')
wg_freq = widgets.FloatSlider(value=200.0,step=5.0,min=50.0,max=500.0,description='Frequency')
wg_Tmax = widgets.FloatSlider(value=0.4,step=0.1,min=0.1,max=2.0,description='Duration',
                             continuous_update=False)
#wg_check = widgets.Checkbox(description='Display Play Button',value=True)
wg_audio = widgets.Output()
wg_plot = widgets.Output()
with wg_plot:
    fig = plt.figure(figsize=(5,4))
first_time = True

# in this example we use interact and full replotting
# this seems to work well with ALL backends

@interact(t=wg_signal,f=wg_freq,A=wg_amp,T=wg_Tmax)
def plot_fn(t,f,A,T):
    global first_time
    x1, t1 = generate(freq=f,type=t,Tmax=T)
    y1=A*x1
    with wg_plot:
        plt.cla()
        plt.plot(t1,y1)
        plt.ylim(-1,1)
    with wg_audio:
        ipd.clear_output()
        ipd.display(ipd.Audio(data=y1,rate=samplerate,normalize=False, autoplay=True))
    return

#widgets.HBox( [widgets.VBox([wg_signal,wg_freq,wg_amp,wg_Tmax]),widgets.VBox([wg_plot,wg_audio])])
widgets.Box( [wg_plot,wg_audio] )

interactive(children=(Dropdown(description='Signal', options=('sin', 'square', 'sawtooth'), value='sin'), Floa…

Box(children=(Output(), Output()))

# 2. Using observe() and callbacks for more control over the interaction
## The Output Widget, Widget Boxes, Observers, Classes, ... and fragile backends

In [17]:
# create some x data and plot it in the output widget
# give control over data, axis, .. via control widgets using callbacks
#
x = np.linspace(0, 2 * np.pi, 100)
output = widgets.Output() 
with output:
    fig, ax = plt.subplots(constrained_layout=True, figsize=(4, 4))
# move the toolbar to the bottom
fig.canvas.toolbar_position = 'bottom'
ax.grid(True)    
line, = ax.plot(x, np.sin(x), '#FF0000')

# create some control elements
int_slider = widgets.IntSlider(value=1, min=0, max=10, step=1, description='freq')
color_picker = widgets.ColorPicker(value='#FF0000', description='pick a color')
text_xlabel = widgets.Text(value='', description='xlabel', continuous_update=False)
text_ylabel = widgets.Text(value='', description='ylabel', continuous_update=False)
 
# callback functions
def update(change):
    """redraw line (update plot)"""
    line.set_ydata(np.sin(change.new * x))
    fig.canvas.draw()
     
def line_color(change):
    """set line color"""
    line.set_color(change.new)
     
def update_xlabel(change):
    ax.set_xlabel(change.new)
     
def update_ylabel(change):
    ax.set_ylabel(change.new)
 
# connect callbacks and traits
int_slider.observe(update, 'value')
color_picker.observe(line_color, 'value')
text_xlabel.observe(update_xlabel, 'value')
text_ylabel.observe(update_ylabel, 'value')
 
text_xlabel.value = 'x'
text_ylabel.value = 'y'
controls = widgets.VBox([int_slider, color_picker, text_xlabel, text_ylabel])
widgets.HBox([controls, output])

## Packing components in a class
### Leveraging the Output widget, observers, classes, ..
### Dedicated to the 'widget' mpl backend

This is a copy from the Kapernikov tutorial ( https://kapernikov.com/ipywidgets-with-matplotlib/ )

To create more high level components, we can also subclass a container and build up our gui from within. Containers have a `children` property to which we can assign a list of widgets that should be displayed. Although, we can assign a list, this is turned into a tuple and cannot be modified afterwards.
To remove or add a widget at runtime, the children tuple can be turned back into a list, followed by an insert or deletion and finalised by reassigning to the children property. Since it can be easy to make mistakes when going by index, I tend to add a placeholder box in which I only place the 'dynamic' widget.

The example below, packs the entire oscilloscope 'dashboard' in a single component by subclassing `VBox`. All the required widgets are defined in the Sines class and added as its `children`. The callbacks are defined as instance methods. It may not be a masterpiece in object oriented programming, but hopefully it shows the idea of constructing larger reusable components. Note that, we need to call `super().__init__()` from `__init__` to properly initialise the parent class. 

In [18]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

class Sines(widgets.HBox):
    
    def __init__(self):
        super().__init__()
        output = widgets.Output()

        self.x = np.linspace(0, 2 * np.pi, 100)
        initial_color = '#FF00DD'

        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(5, 3.5))
        self.line, = self.ax.plot(self.x, np.sin(self.x), initial_color)
        
        self.fig.canvas.toolbar_position = 'bottom'
        self.ax.grid(True)

        # define widgets
        int_slider = widgets.IntSlider(
            value=1, 
            min=0, 
            max=10, 
            step=1, 
            description='freq'
        )
        color_picker = widgets.ColorPicker(
            value=initial_color, 
            description='pick a color'
        )
        text_xlabel = widgets.Text(
            value='', 
            description='xlabel', 
            continuous_update=False
        )
        text_ylabel = widgets.Text(
            value='', 
            description='ylabel', 
            continuous_update=False
        )

        controls = widgets.VBox([
            int_slider, 
            color_picker, 
            text_xlabel, 
            text_ylabel
        ])
        controls.layout = make_box_layout()
        
        out_box = widgets.Box([output])
        output.layout = make_box_layout()

        # observe stuff
        int_slider.observe(self.update, 'value')
        color_picker.observe(self.line_color, 'value')
        text_xlabel.observe(self.update_xlabel, 'value')
        text_ylabel.observe(self.update_ylabel, 'value')
        
        text_xlabel.value = 'x'
        text_ylabel.value = 'y'
        

        # add to children
        self.children = [controls, output]
    
    def update(self, change):
        """Draw line in plot"""
        self.line.set_ydata(np.sin(change.new * self.x))
        self.fig.canvas.draw()

    def line_color(self, change):
        self.line.set_color(change.new)

    def update_xlabel(self, change):
        self.ax.set_xlabel(change.new)

    def update_ylabel(self, change):
        self.ax.set_ylabel(change.new)
        
        
Sines()

Sines(children=(VBox(children=(IntSlider(value=1, description='freq', max=10), ColorPicker(value='#FF00DD', de…

In [19]:
# Method 2: CLEAR selected axes and redraw with each operation (use OO API)
# BACKENDS: NOT for INLINE !
def my_sine(x, w, amp, phi):
    return amp*np.sin(w * (x-phi))
#
x = np.linspace(0, 2 * np.pi, 100)
fig= plt.figure(figsize=(3, 3))

@interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi, 0.01))
def update(w = 1.0, amp=1, phi=0):
    plt.clf()
    gs = fig.add_gridspec(2, 1)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[1, 0])
    #ax = fig.add_axes()
    ax1.plot(x, my_sine(x, w, amp, phi), color='C0')
    ax1.set_ylim([-4, 4])
    ax1.grid(True)
    #plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='w', max=10), FloatSlider(value=1.0, description='amp', m…

In [20]:
firstRun=True
def gui3(number):
    global firstRun
    if firstRun:
        print('number')
        firstRun=False
    print(str(number))

In [21]:
gui3(1)
gui3(50)
gui3(7)

number
1
50
7
